In [6]:
import controller.ml_controller as ml
MODEL_SAVE_DIR = r'..\data\ml_models\trained'
LOG_DIR = r'..\data\ml_models\logs'
DB_PATH = r"..\data\gui_database.db"

In [7]:
# Aucune des variables ici n'est utilisé. Les valeurs ont été implementées en input des fonctions
# --- Chemins et Noms ---
ANONY_IMAGES_PATH = 'data/reconstructed_pipeline'
MODEL_NAME = 'simple_cnn_lfw_anony_v1'
# --- Paramètres des Données et Prétraitement ---
COLOR_MODE = 'grayscale'
IMG_WIDTH, IMG_HEIGHT, CHANNELS = (100, 100, 1 if COLOR_MODE == 'grayscale' else 3)
# --- Paramètres de Division des Données ---
SPLIT_STRATEGY = 'stratified'
# -- Pour 'stratified' --
TEST_SPLIT_RATIO = 0.2
VALIDATION_SPLIT_RATIO = 0.15
# -- Pour 'fixed_per_subject' --
N_TRAIN_PER_SUBJECT = 16
RANDOM_STATE = 42
# ---    Paramètres du Modèle ---
MODEL_ARCHITECTURE = 'simple_cnn'
# -- Pour Transfer Learning (si MODEL_ARCHITECTURE commence par 'transfer_') --
TRANSFER_BASE_MODEL_NAME = 'MobileNetV2'
TRANSFER_FREEZE_BASE = True
# --- Paramètres d'Entraînement ---
EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EARLY_STOPPING_PATIENCE = 10

### Data from folder (LFW dataset)

In [2]:
import os
RECONSTRUCTED_DIR = r"..\data\reconstructed_pipeline"
os.makedirs(RECONSTRUCTED_DIR, exist_ok=True)

X, y, label_encoder = ml.data_loader.load_anonymized_images_flat(
    data_dir=RECONSTRUCTED_DIR,
    img_width=IMG_WIDTH,
    img_height=IMG_HEIGHT,
    color_mode='grayscale'
)

NameError: name 'ml' is not defined

### Data from database

In [8]:
# Get data from db
X, y, label_encoder = ml.get_and_prepare_data(DB_PATH)
input_shape = X[0].shape ;# pas bon - use this insted: IMG_WIDTH, IMG_HEIGHT, CHANNELS
#input_shape = (IMG_WIDTH, IMG_HEIGHT, CHANNELS)
print(X.shape)

(467, 100, 100, 3)


In [9]:
# Create the model
model = ml.train_model(X, y, label_encoder, input_shape=(input_shape), model_save_dir=MODEL_SAVE_DIR, log_dir=LOG_DIR)

Division des données : test_size=0.2, validation_size=0.15
Taille Test: 94 échantillons
Taille Entraînement: 303 échantillons
Taille Validation: 70 échantillons
Construction du modèle CNN simple avec input_shape=(100, 100, 1) et num_classes=43
Modèle CNN simple construit.



Erreur pendant l'entraînement : Exception encountered when calling Functional.call().

Input 0 of layer "conv1_1" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (None, 100, 100, 3)

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(None, 100, 100, 3), dtype=uint8)
  • training=True
  • mask=None

Sauvegarde de l'encodeur de labels (même si l'entraînement a échoué)...
LabelEncoder sauvegardé dans : ..\data\models\trained\simple_cnn_lfw_anony_v1_label_encoder.joblib


In [5]:
print(f"summary_text: {model['summary_text']}")
print(f"duration: {model['duration']}")
print(f"evaluation: {model['evaluation']}")
print(f"training_plot: {model['training_plot']}") # .show()

summary_text: Model: "simple_cnn"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)        │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_1 (Conv2D)                │ (None, 100, 100, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1_1 (BatchNormalization)      │ (None, 100, 100, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_2 (Conv2D)                │ (None, 100, 100, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1_2 (BatchNormalization)      │ (None, 100, 100, 32)   │           128 │
├─────────────────────────────────┼───────

In [9]:
# Image to predict
index = 30

img_to_predict = X[index]
true_result = y[index]
print(img_to_predict.shape)

# Make a prediction
prediction = ml.predict_image(img_to_predict, model_save_dir=MODEL_SAVE_DIR)

# Result
print(f"prediction: {prediction}")
print(f"true_result: {true_result}")
print(f"The model is correct: {prediction==true_result}")

LabelEncoder chargé depuis : ..\data\models\trained\simple_cnn_lfw_anony_v1_label_encoder.joblib
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
prediction: 18
true_result: 2
The model is correct: False
